In [3]:
import numpy as np
from matplotlib import pyplot as plt
import random
from sklearn.naive_bayes import GaussianNB
import math
from collections import Counter

In [53]:
LANE_WIDTH = 4
ROAD_LENGTH = 100
SPEED = 10
NUM_LANES = 3

def generate_labeled_data(N):
    trajectories = []
    labels = []
    for i in range(N):
        start_lane = random.choice([0,1,2])
        label = "keep"
        if start_lane == 0:
            end_lane = random.choice([0,1])
        elif start_lane == 1:
            end_lane = random.choice([0,1,2])
        else:
            end_lane = random.choice([1,2])
        if end_lane > start_lane: 
            label = "right"
        elif end_lane < start_lane:
            label = "left"
        start_s = 0.0
        dist = random.gauss(40, 3)
        end_s = start_s + dist
        start_d = LANE_WIDTH * start_lane + random.gauss(0, float(LANE_WIDTH) / 10)
        end_d  =  LANE_WIDTH * end_lane   + random.gauss(0, float(LANE_WIDTH) / 10)
        speed = max(random.gauss(SPEED, 1.0), 6)
        T = (end_s - start_s) / speed
        alpha_s = JMT([start_s, speed, 0], [end_s, speed + random.gauss(0,0.5), 0], T)
        alpha_d = JMT([start_d, 0, 0], [end_d, 0, 0], T)
        s = to_equation(alpha_s)
        s_dot = to_equation(differentiate(alpha_s))
        d = to_equation(alpha_d)
        d_dot = to_equation(differentiate(alpha_d))
        coords = []
        for i in range(0, int(T*100), int(2*T) ):
            t = float(i) / 100
            coords.append((s(t), d(t), s_dot(t), d_dot(t)))
        trajectories.append(coords)
        labels.append(label)
    return trajectories, labels

def differentiate(coefficients):
    new_cos = []
    for deg, prev_co in enumerate(coefficients[1:]):
        new_cos.append((deg+1) * prev_co)
    return new_cos


N = 100
trajs, labels = generate_labeled_data(N)
TRAIN_DATA = trajs[:3 * N / 4]
TRAIN_LABELS = labels[:3 * N / 4]
TEST_DATA  = trajs[3 * N / 4:]
TEST_LABEL = labels[3 * N / 4:]

write_to_file( TRAIN_DATA,TRAIN_LABELS, 'train.txt')

T = 15
def process_obs(obs):
    s,d,s_dot, d_dot = obs
    return (
        s,
#             (d % LANE_WIDTH) ** 2,
#             d_dot ** 2,
#             abs(d),
#             d/d_dot,
            )
def train_naive_bayes(data,labels, t):
    X = []
    transform_label = {'left' : -1, 'keep' : 0, 'right' : 1}
    Y = [transform_label[lab] for lab in labels]
    for trajectory in data:
        obs = trajectory[t]
        X.append(process_obs(obs))
    clf = GaussianNB()
    clf.fit(X, Y)
    return clf

# clf = train_naive_bayes(TRAIN_DATA, TRAIN_LABELS, T)

[  9.67513866e-04  -2.43729837e-04   1.12527259e-06]
[  3.22179007e-02  -6.06905690e-03   1.89156037e-05]
[  4.52531233e-02  -9.32395917e-03   2.38516155e-05]
[  1.03994117e-01  -1.60376227e-02   2.75817588e-05]
[  8.71976564e-04  -2.18025355e-04   9.84811922e-07]
[  2.81414515e-01  -5.26185646e-02   1.60418881e-04]
[  2.22732416e-02  -4.20260288e-03   8.29027921e-06]
[ -8.65960598e-03   1.22334623e-03  -1.62023891e-06]
[ -3.60487941e-02   1.01739847e-02  -6.54148693e-05]
[ -3.58740690e-01   7.56576746e-02  -3.29422820e-04]
[  2.86591453e-02  -6.19690271e-03   1.82747323e-05]
[  1.60574738e-01  -2.59826336e-02   5.15573919e-05]
[  1.58267639e-02  -3.76340203e-03   1.46737271e-05]
[ -1.95952263e-01   3.48532360e-02  -9.16150263e-05]
[ 0.0470653  -0.01581401  0.0001682 ]
[ 0.07866339 -0.01972444  0.00014298]
[ -1.71364721e-02   4.86057190e-03  -3.17075106e-05]
[ -3.61442223e-01   7.66059200e-02  -3.38469448e-04]
[ -2.70300029e-02   5.94169678e-03  -1.83923859e-05]
[  1.69700131e-01  -2.7

In [1]:
def JMT(start, end, T):
    a_0, a_1, a_2 = start[0], start[1], start[2] / 2.0
    c_0 = a_0 + a_1 * T + a_2 * T**2
    c_1 = a_1 + 2 * a_2 * T
    c_2 = 2 * a_2
    
    A = np.array([
            [  T**3,   T**4,    T**5],
            [3*T**2, 4*T**3,  5*T**4],
            [6*T,   12*T**2, 20*T**5],
        ])
    B = np.array([
            end[0] - c_0,
            end[1] - c_1,
            end[2] - c_2
        ])
    a_3_4_5 = np.linalg.solve(A,B)
    print a_3_4_5
    alphas = np.concatenate([np.array([a_0, a_1, a_2]), a_3_4_5])
    return alphas

def to_equation(coefficients):
    def f(t):
        total = 0.0
        for i, c in enumerate(coefficients): 
            total += c * t ** i
        return total
    return f

def PTG(start_s, start_d, end_s, end_d, T, time_step = 0.1):
    s_coefficients = JMT(start_s, end_s, T)
    d_coefficients = JMT(start_d, end_d, T)
    print "s_coefficients: {}".format(s_coefficients)
    s = to_equation(s_coefficients)
    d = to_equation(d_coefficients)
    coords = []
    t = 0
    while t <= T:
        new = (s(t), d(t), t)
        coords.append(new)
        t += time_step
    return coords

In [73]:
cnt = Counter()
for l in TRAIN_LABELS + TEST_LABEL:
    cnt[l] += 1
    

In [74]:
class Predictor():
    def __init__(self):

Counter({'keep': 464, 'left': 253, 'right': 283})

In [71]:
transform_label = {'left' : -1, 'keep' : 0, 'right' : 1}
Y = [transform_label[l] for l in TEST_LABEL]
X = [process_obs(td[T]) for td in TEST_DATA]
clf.score(X, Y)

0.46400000000000002

In [66]:
clf.predict_proba(X[20])

array([[  3.04824214e-04,   0.00000000e+00,   9.99695176e-01]])

In [67]:
Y[20]

1

In [ ]:
COL = {'keep' : 'black', 'left'  : 'red', 'right':'blue'}

for coords, label in zip(trajs, labels):
    S = [p[0] for p in coords]
    D = [p[1] for p in coords]
    S_DOT = [p[2] for p in coords]
    D_DOT = [p[3] for p in coords]
    print label
    color = COL[label]
    plt.scatter(S,D,color=color)
plt.show()
                         

keep
right
right
keep
left
right
left
keep
left
right
keep
keep
keep
left
right
keep
left
left
left
keep
keep
left
right
left
keep
keep
keep
right
right
keep
left
keep
right
keep
left
keep
left
right
right
left
right
right
keep
keep
keep
left
keep
keep
keep
keep
keep
right
left
right
keep
right
right
right
left
left
left
right
keep
keep
right
right
left
left
keep
keep
left
left
left
right
right
right
right
right
right
keep
right
right
right
keep
keep
keep
right
keep
keep
right

In [9]:
coords = generate_lane_change(1,0)
X = [p[0] for p in coords]
Y = [p[1] for p in coords]

[ 0.  0. -0.]
[ -1.28796680e-01   1.95186722e-02  -3.18672199e-05]


In [10]:
plt.scatter(X,Y)

In [11]:
plt.show()

//anaconda/lib/python2.7/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [38]:
import json
def to_data_points(trajs):
    return  [random.choice(coords) for coords in trajs]

def write_to_file(data, labels, fname):
    with open(fname, 'wb') as f:
        j = {
            "states" : to_data_points(data) ,
            "labels" : labels
        }
        json.dump( j, f)

In [37]:
to_data_points(TRAIN_DATA[:2])

[(21.42566959631317,
  3.0651493987571969,
  8.4057187042559853,
  -1.3666334365586592),
 (9.7370256001445945,
  4.1120139942043048,
  7.7543823315803095,
  0.34130366666833728)]

In [20]:
TRAIN_DATA[0]

[(0.0, 4.5708033702766695, 8.3260292586807569, 0.0),
 (0.33304208214912617,
  4.5707937842828468,
  8.326097448705001,
  -0.00071741753390899615),
 (0.66608957267885738,
  4.5707271725301633,
  8.3262988975006866,
  -0.0028451613125867678),
 (0.99914770833224309,
  4.5705478570724321,
  8.3266289251131695,
  -0.0063464784570837467),
 (1.3322215387064382,
  4.570201629802475,
  8.3270828542023612,
  -0.011184629895949373),
 (1.6653159263572859,
  4.5696357518998214,
  8.3276560100427481,
  -0.017322890365232095),
 (1.9984355469038995,
  4.5687989532784066,
  8.3283437205233746,
  -0.024724548408479357),
 (2.3315848891332456,
  4.5676414320342786,
  8.3291413161478474,
  -0.033352906376737636),
 (2.6647682551047254,
  4.5661148538932901,
  8.3300441300343415,
  -0.043171280428552387),
 (2.9979897602547605,
  4.5641723516588035,
  8.3310474979155913,
  -0.054143000529968094),
 (3.3312533335013716,
  4.5617685246593869,
  8.3321467581389026,
  -0.066231410454528253),
 (3.6645627173487627,


In [39]:
write_to_file(TRAIN_DATA, TRAIN_LABELS, 'train.json')
write_to_file(TEST_DATA, TEST_LABEL, 'test.json')

In [40]:
with open('train.json', 'rb') as f:
    j = json.load(f)

In [41]:
len(j)

2

In [43]:
len(j['states'])

750

In [44]:
j['states'][0]

[34.76807295828066, 0.8329138124321809, 8.206638634642987, -0.9989611424577053]